In [2]:
import pandas as pd
import pyupbit
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

def define_crypto(crypto, to_date, count):
    df = pyupbit.get_ohlcv(crypto, interval="day", to=to_date, count=count, period=0.1)
    df = df.reset_index()
    df = df.rename(columns={'index': 'date'})
    df['date'] = df['date'].dt.to_period(freq='D')
    df = df.set_index('date')
    return df

dataframes = {}
crypto_names = []

cols = ['KRW-SOL','KRW-XRP','KRW-ETH','KRW-TRX','KRW-BTC','KRW-LINK',
        'KRW-HBAR','KRW-ETC','KRW-DOGE','KRW-ALGO','KRW-BSV','KRW-EOS',
        'KRW-DOT','KRW-FLOW','KRW-ADA','KRW-SC','KRW-SAND','KRW-UPP','KRW-QTUM']

for crypto in cols:
    crypto_name = crypto.split('-')[1]
    df = define_crypto(crypto, datetime.today().strftime("%Y%m%d"), 365)
    dataframes[crypto_name] = df
    globals()[crypto_name] = dataframes.get(crypto_name)
    
    if crypto_name not in crypto_names:
        crypto_names.append(crypto_name)

In [52]:
pd.options.display.float_format = '{:.1f}'.format
iloc_ranges = [-30, -50, -100, -200, -365]
rolling_windows = [5, 10, 15]

filter_buy_df = pd.DataFrame(columns=['crypto', '(n)ma', 'date', 'close', 'ma'])

for crypto_name in crypto_names:
    for iloc_range in iloc_ranges:
        for rolling_window in rolling_windows:
            df = globals()[crypto_name][['close']].iloc[iloc_range:, :].copy()
            
            df['MA'] = df['close'].rolling(rolling_window).mean().shift(1)
            df['action'] = np.where(df['close'] > df['MA'], 'buy', 'sell')
            cond_buy = (df['action'] == 'buy') & (df['action'].shift(1) == 'sell')
            
            df_buy = df[cond_buy].reset_index()
            df_buy.columns = ['date(buy)', 'close(buy)', 'ma', 'action(buy)']
            
            if not df_buy.empty:
                last_date_buy = df_buy.tail(1)['date(buy)'].values[0]
                last_date_buy_datetime = last_date_buy.to_timestamp()
                compare_date = datetime.today() - timedelta(1)
                
                if last_date_buy_datetime.date() == compare_date.date():
                    buy_row = pd.DataFrame({'crypto': [crypto_name],
                                            '(n)ma': [rolling_window],
                                            'date': [last_date_buy],
                                            'close': [df_buy.tail(1)['close(buy)'].values[0]],
                                            'ma': [df['close'].rolling(rolling_window).mean().iloc[-1]] })
                    filter_buy_df = pd.concat([filter_buy_df, buy_row], ignore_index=True)
                    filter_buy_df = filter_buy_df.drop_duplicates(['crypto','(n)ma'], keep='last').reset_index(drop=True)                
                    
filter_buy_df

,crypto,(n)ma,date,close,ma
0,XRP,5,2024-07-30,879.5,846.4
1,UPP,5,2024-07-30,77.3,77.6


In [53]:
iloc_ranges = [-30, -50, -100, -200, -365]
rolling_windows = [5, 10, 15]

filter_sell_df = pd.DataFrame(columns=['crypto', '(n)ma', 'date', 'close', 'ma'])

for crypto_name in crypto_names:
    for iloc_range in iloc_ranges:
        for rolling_window in rolling_windows:
            df = globals()[crypto_name][['close']].iloc[iloc_range:,:].copy()

            df['MA'] = df['close'].rolling(rolling_window).mean().shift(1)
            df['action'] = np.where(df['close'] > df['MA'], 'buy', 'sell')
            cond_sell = (df['action'] == 'sell') & (df['action'].shift(1) == 'buy')

            df_sell = df[cond_sell].reset_index()
            df_sell.columns=['date(sell)','close(sell)','ma','action(sell)']

            if not df_sell.empty:
                last_date_sell = df_sell.tail(1)['date(sell)'].values[0]
                last_date_sell_datetime = last_date_sell.to_timestamp()
                compare_date = datetime.today() - timedelta(1)

                if last_date_sell_datetime.date() == compare_date.date():
                    sell_row = pd.DataFrame({'crypto': [crypto_name],
                                            '(n)ma': [rolling_window],
                                            'date': [last_date_sell],
                                            'close': [df_sell.tail(1)['close(sell)'].values[0]],
                                            'ma': [df['close'].rolling(rolling_window).mean().iloc[-1]] })
                    filter_sell_df = pd.concat([filter_sell_df, sell_row], ignore_index=True)
                    filter_sell_df = filter_sell_df.drop_duplicates(['crypto','(n)ma'], keep='last').reset_index(drop=True)

filter_sell_df

,crypto,(n)ma,date,close,ma
0,SOL,5,2024-07-30,251050.0,255040.0
1,TRX,5,2024-07-30,186.5,191.1
2,TRX,10,2024-07-30,186.5,189.4
3,TRX,15,2024-07-30,186.5,188.9
4,BTC,15,2024-07-30,92788000.0,92902266.7
5,LINK,5,2024-07-30,18410.0,18732.0
6,ETC,5,2024-07-30,31180.0,31692.0
7,DOGE,15,2024-07-30,176.0,180.6
8,SC,5,2024-07-30,7.2,7.5
9,SC,10,2024-07-30,7.2,7.4


In [66]:
iloc_ranges = [-30, -50, -100, -200, -365]
rolling_windows = [5, 10, 15]

filter_return_df = pd.DataFrame(columns=['crypto', '(n)ma', 'day', 'date', 'return(%)'])

for crypto_name in crypto_names:
    for iloc_range in iloc_ranges:
        for rolling_window in rolling_windows:
            df = globals()[crypto_name][['close']].iloc[iloc_range:,:].copy()

            df['MA'] = df['close'].rolling(rolling_window).mean().shift(1)
            df['action'] = np.where(df['close'] > df['MA'], 'buy', 'sell')
            cond_buy = (df['action'] == 'buy') & (df['action'].shift(1) == 'sell')
            cond_sell = (df['action'] == 'sell') & (df['action'].shift(1) == 'buy')
            df.iloc[-1,-1] = 'sell'

            df_buy = df[cond_buy].reset_index()
            df_buy.columns=['date(buy)','close(buy)','ma','action(buy)']
            df_sell = df[cond_sell].reset_index()
            df_sell.columns=['date(sell)','close(sell)','ma','action(sell)']

            df_result = pd.concat([df_buy, df_sell], axis=1)
            df_result['return_rate'] = df_result['close(sell)'] / df_result['close(buy)']
            df_result = df_result.dropna()
            
            if not df_result.empty:
                final_return = round((df_result[['return_rate']].cumprod().iloc[-1, -1] - 1) *100, 1)
                return_row = pd.DataFrame({'crypto': [crypto_name],
                                            '(n)ma': [rolling_window],
                                            'day': [-iloc_range],
                                            'date': [(datetime.today() - timedelta(days=1)).strftime("%Y-%m-%d")],
                                            'return(%)': [f"{final_return:.1f}"] })
                filter_return_df = pd.concat([filter_return_df, return_row], ignore_index=True)
                filter_return_df = filter_return_df.astype({'(n)ma':int, 'day':int})
                filter_return_df['return(%)'] = filter_return_df['return(%)'].apply(lambda x: round(float(x), 1)).astype(float)              
                filter_return_df['date'] = pd.to_datetime(filter_return_df['date'])
                
filter_return_df

,crypto,(n)ma,day,date,return(%)
0,SOL,5,30,2024-07-30,7.8
1,SOL,5,50,2024-07-30,5.1
2,SOL,10,50,2024-07-30,-16.1
3,SOL,15,50,2024-07-30,-22.4
4,SOL,5,100,2024-07-30,-2.0
...,...,...,...,...,...
274,QTUM,10,200,2024-07-30,11.6
275,QTUM,15,200,2024-07-30,-7.2
276,QTUM,5,365,2024-07-30,8.2
277,QTUM,10,365,2024-07-30,-13.4


In [72]:
show_return = filter_return_df.groupby(['crypto','(n)ma','day'])[['return(%)']].mean().sort_values('return(%)', ascending=False).reset_index()
show_return[show_return['day'] == 30]

,crypto,(n)ma,day,return(%)
55,XRP,5,30,36.8
64,XRP,10,30,30.6
80,BSV,5,30,20.3
86,QTUM,5,30,17.3
94,DOGE,5,30,14.4
95,SC,5,30,14.4
100,BSV,10,30,12.9
102,BTC,10,30,11.8
104,UPP,5,30,11.6
106,QTUM,10,30,11.4


In [ ]:
# 7/30 퇴근하고 할것
# 1.최근 n개 데이터, 이평선값 바꿔서도 출력되도록 하기. (최근 한달, 최근 50일, 최근 100일, 최근 200일, 최근 365일) --> 완료
# 2.결과값을 데이터프레임으로 만들기 / 수익률도 데이터프레임 & 시각화
# 3.3가지 내용을 합쳐서 Dash / Airflow 구축하기 --> 너가 몇월 몇일에 매수를 해서 이 백테스팅 룰을 그대로 지켰다면 현재 수익률은?